In [ ]:
!pip install xgboost lightgbm pandas seaborn scikit-learn optuna expetator cryptography

In [ ]:
!pip install imbalanced-learn

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/ML_DVFS/Modeling and evaluation'
os.chdir(PATH)

Mounted at /content/drive


In [3]:
from expetator.tools import read_experiment, show_heatmap, add_objectives
from expetator.tools import prune_vectors, mojitos_to_vectors, show_pct_distribution
from expetator.monitors import mojitos
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, f1_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import xgboost as xgb
import lightgbm as lgb
import multiprocessing
#import optuna.integration.lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from random import sample 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from functools import partial
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
def generate_en_features(df):
    df = df.copy()
    cols = set(df.columns)-{'metric','expe','group','target'}
    for col in cols:
        for func in ['mean','std','min','max']:
            temp = df.groupby('expe')[col].agg([func]).rename({func:f"{col}_{func}"},axis=1)
            df = pd.merge(df,temp,on='expe',how='left')
        df[col+'_bin10'] = pd.cut(df[col], bins = 10, labels = False)
        df[col+'_bin100'] = pd.cut(df[col], bins = 100, labels = False)
        df[col+'_log'] = df[col].apply(lambda row: np.log(1+row))
        df[col+'log+1'] = (df[col]+1).transform(np.log)
        df[col+'log(x-min(x)+1)'] = (df[col]-df[col].min()+1) .transform(np.log)
    return df
def generate_aggregations(df):
    df = df.copy()
    for col in set(df.columns)-{'metric','expe','freq','group','target'}:
        for func in ['mean','std','min','max']:
          temp = vectors.groupby('expe')[col].agg([func]).rename({func:f"{col}_{func}"},axis=1)
          df = pd.merge(df,temp,on='expe',how='left')
    return df
def outlierdrop(df,factor):
    for col in set(df.columns) - {'metric','expe','freq','group','target'}:
        upper_lim = df[col].mean () + df[col].std () * factor
        lower_lim = df[col].mean () - df[col].std () * factor
        df = df[(df[col] < upper_lim) & (df[col] > lower_lim)]
    return df
def outlierfill(df):
    for column in set(df.columns) - {'metric','expe','freq','group','target'}:
        upper_lim = df[column].quantile(.95)
        lower_lim = df[column].quantile(0.05)
        df.loc[(df[column] > upper_lim),column] = upper_lim
        df.loc[(df[column] < lower_lim),column] = lower_lim
    return df
all_knowledge = pd.read_csv("../csvs/knowledge_allmetrics.csv")
all_vectors = pd.read_csv("../csvs/vectors_allmetrics.csv")
all_vectors['typedata'] = 0

all_knowledge_old = pd.read_csv("../csvs/old/knowledge_allmetrics.csv")
all_vectors_old = pd.read_csv("../csvs/old/vectors_allmetrics.csv")
all_knowledge_old['expe'] = all_knowledge_old['expe'] + all_knowledge.expe.values[-1] + 1
all_vectors_old['typedata'] = 1

all_knowledge = pd.concat([all_knowledge, all_knowledge_old],ignore_index=True)
all_vectors = pd.concat([all_vectors, all_vectors_old],ignore_index=True)

# merge knowlage to vectors => data for training 
all_vectors['expe'] = all_knowledge.expe
all_vectors['target'] = all_knowledge['target']

ignore_columns = ['irxp', 'irxb','itxp', 'itxb', 'core0','core1','bpf_output',
                 'alignment_faults','page_faults_maj','dummy','emulation_faults','nice',
                 'irq','steal','guest','guest_nice']
all_vectors = all_vectors.loc[:,~ all_vectors.columns.isin(ignore_columns)]
all_vectors_outlier = outlierfill(all_vectors)
vectorbin = generate_en_features(all_vectors_outlier)
vectors = prune_vectors(vectorbin)


ignore_columns = ['typedata','expe','group','target','Kfold', #'guest', 'guest_nice', 'irq',
                  #'steal','nice','emulation_faults','irxp', 'irxb', 
                  #'itxp', 'itxb', 'core0','core1','iowait','softirq','txp',
                 ]

# convert categories to numerical
metric_mapping = {
    "energy":0,
    "duration":1,
    "etp":2
}
#vectors['metric'] = all_knowledge['metric'].map(metric_mapping)
vectors = vectors[all_knowledge['metric'] == 'duration']
labels = [1200000,
     1300000,
     1400000,
     1500000,
     1600000,
     1700000,
     1800000,
     1900000,
     2000000,
     2100000,
     2200000,
     2300000,
     2400000]
vectors['target'] = vectors.target.apply(lambda row: labels.index(row))

#split dataset to train, test: 0.8:0.2. Note that split base on expe
testidx = []
exps = list(set(vectors[vectors['typedata'] == 1].expe))
testidx.extend(sample(exps, int(0.2*len(exps))))

exps = list(set(vectors[vectors['typedata'] == 0].expe))
testidx.extend(sample(exps, int(0.2*len(exps))))

train = vectors[~vectors.expe.isin(testidx)].reset_index(drop=True)
test = vectors[vectors.expe.isin(testidx)].reset_index(drop=True)

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [9]:
def lgb_f1mse_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1', f1_score(y_true, y_hat) + 2*mean_squared_error(y_true,y_hat), True
metric_list=['merror']
earlyStop=30
def optimize(trial):
    boosting_list = ['gbdt','goss']
    objective_list_reg = ['huber', 'gamma', 'fair', 'tweedie']
    objective_list_class = ['multiclass','multiclassova']
    param={
      'num_class':13,
      'boosting':trial.suggest_categorical('boosting',boosting_list),
      'num_leaves':trial.suggest_int('num_leaves', 2, 2**11),
      'max_depth':trial.suggest_int('max_depth', 2, 25),
      'max_bin': trial.suggest_int('max_bin', 32, 255),      
      'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 256),
      'min_data_in_bin': trial.suggest_int('min_data_in_bin', 1, 256),
      'min_gain_to_split' : trial.suggest_discrete_uniform('min_gain_to_split', 0.1, 5, 0.01),      
      'lambda_l1':trial.suggest_loguniform('lamda_l1',1e-8,10),
      'lambda_l2':trial.suggest_loguniform('lamda_l2',1e-8,10),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
      'metric':trial.suggest_categorical('metric', ['multi_logloss','multi_error']),#'multi_logloss','multi_error'
      'objective':trial.suggest_categorical('objective',objective_list_class),
      'bagging_fraction':trial.suggest_discrete_uniform('bagging_fraction',0.5, 1, 0.01),
      'feature_fraction':trial.suggest_discrete_uniform('feature_fraction',0.5, 1, 0.01),
      'num_threads': multiprocessing.cpu_count(),
      'verbose':-1,
      #'is_unbalance':'true',
      #'class_weight':class_weight_dict
        
    }
    
    
    accuracies = []
    skf = GroupKFold(n_splits=5)
    train['group'] = train.apply(lambda row: str(row['expe'])+str(row['typedata']),axis = 1)
    test_pro = []
    for i, (train_index, test_index) in enumerate(skf.split(train, train.target, groups = train['group'])):
        xtrain_fold = train.loc[train_index, ~train.columns.isin(ignore_columns)]
        ytrain_fold = train.loc[train_index,'target']
        X_val_fold = train.loc[test_index, ~train.columns.isin(ignore_columns)]
        y_val_fold = train.loc[test_index,'target']
        

        df_Ytrain = ytrain_fold.reset_index(drop=True)
        df_Yval = y_val_fold.reset_index(drop=True)

        sc = StandardScaler()
        df_train = sc.fit_transform(xtrain_fold)
        df_val = sc.transform(X_val_fold)
    
        #over = SMOTE()
        #under = RandomUnderSampler()
        #steps = [('u', under)]#('o', over), 
        #pipeline = Pipeline(steps=steps)
        #xthang, ythang = pipeline.fit_resample(df_train, df_Ytrain)

        #df_Xtrain_sc = pd.DataFrame(xthang,columns = xtrain_fold.columns)
        df_Xtrain_sc = pd.DataFrame(df_train,columns = xtrain_fold.columns)
        df_Xval_sc = pd.DataFrame(df_val,columns = X_val_fold.columns)

        
        dtrain = lgb.Dataset(df_Xtrain_sc,label=df_Ytrain)
        dval = lgb.Dataset(df_Xval_sc,label=df_Yval)
        watchlist = [dtrain,dval]
        lgbmPruningCallback = optuna.integration.LightGBMPruningCallback(trial, 'rmse', valid_name='valid_2')

        model = lgb.train(param,train_set=dtrain,num_boost_round=1000,valid_sets=watchlist,verbose_eval=-1,early_stopping_rounds=earlyStop)#callbacks=[lgbmPruningCallback]
        #bst = lgb.train(param, dtrain)
        #preds = bst.predict(dval)
        pred_val=model.predict(df_Xval_sc,num_iteration=model.best_iteration)
        
        df_Yval = y_val_fold.reset_index(drop=True)
        preds = np.argmax(pred_val,axis = 1)
        accuracy = accuracy_score(df_Yval, preds)
        print(f"                                                                                  FOLD{i}-val acc: {(accuracy * 100.0)}", end=' ')
        
        accuracies.append(accuracy)
        
        X_test = test.loc[:, ~test.columns.isin(ignore_columns)]
        df_Xtest = sc.transform(X_test)
        df_Xtest_sc = pd.DataFrame(df_Xtest,columns = X_test.columns)
        y_test = test.target.reset_index(drop=True)
        pred_val=model.predict(df_Xtest_sc,num_iteration=model.best_iteration)
        
        #dtest = lgb.Dataset(df_Xtest_sc,label=y_test)
        test_pro.append(pred_val)
        #preds = bst.predict(dtest)
        preds = np.argmax(pred_val,axis = 1)
        accuracy = accuracy_score(y_test, preds)
        print(f" FOLD{i}-test acc: {(accuracy * 100.0)}")
    test_pro_mean = np.mean(test_pro,axis=0)
    preds = np.argmax(test_pro_mean,axis = 1)
    accuracy = accuracy_score(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    mse_fre = mean_squared_error([labels[i] for i in y_test], [labels[i] for i in preds])
    print(f"=====================> TTA on test acc: {(accuracy * 100.0)}, mse: {mse} ~ {mse_fre}")
    
    if accuracy > 0.77:
      print(classification_report(y_test, preds))
      cfm = confusion_matrix(y_test, preds, labels = range(0,13))
      cm = cfm
      cfm = cfm.astype('float') / cfm.sum(axis=1)[:, np.newaxis]
      tmplabel = [i//100000 for i in labels]
      annot = np.empty_like(cfm).astype(str)
      nrows, ncols = cfm.shape
      for i in range(nrows):
          for j in range(ncols):
              c = cfm[i, j]*100
              if c == 0:
                  annot[i, j] = ''
              else:
                annot[i, j] = '%.1f%%\n%d' % (c,cm[i, j])
      plt.figure(figsize=(12,10))
      ax = sns.heatmap(cfm, annot=annot, fmt='',cmap="YlGnBu")
      ax.set_xticklabels(tmplabel)
      ax.set_yticklabels(tmplabel)
      plt.xlabel('Predicted')
      plt.ylabel('True')
      plt.show()       
        
    return -1.0*np.mean(accuracies)

In [10]:
#optimization_function = partial(optimize, X=x, y=y)
study  = optuna.create_study(direction = "minimize")
#study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))

study.optimize(optimize, n_trials=500)
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    

Output hidden; open in https://colab.research.google.com to view.

In [12]:
all_vectors_old

,ref_cpu_cycles,cpu_clock,branch_misses,cache_references,page_faults,context_switches,cpu_cycles,cache_dtlb,cache_op_read,alignment_faults,cache_node,task_clock,bus_cycles,bpf_output,cache_l1d,cache_itlb,branch_instructions,cache_misses,page_faults_maj,cache_ll,cache_bpu,cache_op_prefetch,page_faults_min,cpu_migrations,dummy,cache_result_access,instructions,emulation_faults,rxp,rxb,txp,txb,package-00,dram0,package-11,dram1,user,nice,system,idle,iowait,irq,softirq,steal,guest,guest_nice,irxp,irxb,itxp,itxb,core0,core1,freq,typedata
0,1.171386e+09,3.069360e+09,1.326824e+06,4.036555e+06,194.704156,139.894866,5.240105e+08,1.119943e+08,1.158463e+08,0.0,355321.237164,3.073166e+09,2.616438e+07,0.0,1.090133e+08,48230.716381,2.445892e+07,833957.904645,0.0,2.792697e+06,5.033048e+07,2.758105e+06,191.376528,0.070905,0.0,1.505519e+08,5.658242e+08,0.0,1773.535452,1.566692e+07,1137.381418,1.563859e+07,3.714634e+06,1.055089e+06,3.357083e+06,1.028287e+06,113.938875,0.0,43.479218,157.019560,0.0,0.0,2.577017,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000,1
1,1.178697e+09,3.076990e+09,1.543819e+06,4.275998e+06,230.388471,147.318296,5.752833e+08,1.120118e+08,1.085217e+08,0.0,355653.360902,3.080577e+09,2.590508e+07,0.0,1.137197e+08,42522.335840,2.810298e+07,836379.363409,0.0,2.923861e+06,5.627428e+07,2.950423e+06,227.982456,0.040100,0.0,1.595268e+08,6.049458e+08,0.0,1922.215539,1.613431e+07,1221.308271,1.610461e+07,3.861496e+06,1.064366e+06,3.502719e+06,1.034292e+06,112.383459,0.0,45.218045,157.200501,0.0,0.0,2.466165,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1300000,1
2,1.186990e+09,3.082000e+09,1.667409e+06,4.428871e+06,323.041667,154.054688,6.138289e+08,1.190939e+08,1.206594e+08,0.0,373936.049479,3.085331e+09,2.584308e+07,0.0,1.121994e+08,40794.398438,3.039741e+07,860039.489583,0.0,3.100954e+06,6.045978e+07,3.080752e+06,318.273438,0.145833,0.0,1.711049e+08,6.486680e+08,0.0,2075.989583,1.676075e+07,1332.473958,1.673771e+07,4.011885e+06,1.078241e+06,3.617672e+06,1.043768e+06,111.390625,0.0,46.031250,157.372396,0.0,0.0,2.513021,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1400000,1
3,1.193797e+09,3.087653e+09,1.787191e+06,4.717843e+06,132.236702,157.664894,6.694202e+08,1.449491e+08,1.396374e+08,0.0,372786.710106,3.090455e+09,2.664127e+07,0.0,1.306964e+08,52602.034574,3.277650e+07,897513.691489,0.0,3.277662e+06,6.603291e+07,3.238617e+06,134.752660,0.077128,0.0,1.802622e+08,6.847907e+08,0.0,2170.196809,1.709885e+07,1395.739362,1.707149e+07,4.146436e+06,1.085909e+06,3.766941e+06,1.048668e+06,109.968085,0.0,47.369681,157.428191,0.0,0.0,2.521277,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1500000,1
4,1.138394e+09,3.094404e+09,1.835530e+06,4.475046e+06,164.174863,158.849727,6.668155e+08,1.289870e+08,1.391217e+08,0.0,373482.576503,3.096553e+09,2.502688e+07,0.0,1.304134e+08,47366.379781,3.367439e+07,845190.601093,0.0,3.119317e+06,6.738774e+07,3.164678e+06,162.789617,0.051913,0.0,1.814418e+08,6.872917e+08,0.0,2253.224044,1.753363e+07,1463.377049,1.748716e+07,4.277000e+06,1.092290e+06,3.898124e+06,1.051759e+06,109.226776,0.0,48.098361,157.560109,0.0,0.0,2.420765,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1600000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47818,6.492267e+08,3.122470e+09,8.749247e+05,6.774552e+05,0.002079,107.390852,4.871824e+08,4.375409e+08,4.425976e+08,0.0,326.873181,3.124101e+09,2.375065e+07,0.0,3.135356e+08,28568.488565,2.224796e+08,328756.457380,0.0,5.648993e+05,2.302166e+08,3.953972e+05,0.014553,0.010395,0.0,4.096235e+08,1.293810e+09,0.0,1967.205821,1.446424e+07,1833.844075,1.448326e+07,5.117036e+06,6.296214e+05,4.876645e+06,6.430999e+05,140.070686,0.0,18.790021,158.588358,0.0,0.0,1.345114,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2000000,1
47819,7.653543e+08,3.126823e+09,1.122749e+06,1.003358e+06,0.002165,115.194805,6.177618e+08,5.473929e+08,3.908299e+08,0.0,303.956710,3.128234e+09,3.042641e+07,0.0,